<a href="https://colab.research.google.com/github/Alejom74/Business-Inteligence/blob/main/Reto_4_Alejandro_Mariaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Se procede con la instalación de las librerías de análisis

In [1]:
!pip install vaderSentiment
!pip install -U spacy
!python -m spacy download es_core_news_lg
!pip install download en_core_web_sm
!pip install translate
!pip install wordcloud

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 3.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


1. Se procede con la instalación de los paquetes de trabajo

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk #Natural Language Toolkit
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt_tab') #Para hacerlo más eficiente

from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Mounted at /content/drive


2. Se cargan las librerías de trabajo

In [3]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy
from translate import Translator
from wordcloud import WordCloud

# Cargar modelo de lenguaje en español
nlp = spacy.load("es_core_news_lg")
sia = SentimentIntensityAnalyzer()

3. Análisis Informe Junta 2021-2024

In [4]:
def analizar_sentimientos(archivo, year):
    with open(archivo, 'r', encoding='utf-8') as f:
        texto = f.read()

    frases = sent_tokenize(texto)
    pnst = np.array([-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    nd = np.zeros((11, 1))
    resultados = []

    # Definir las categorías de sentimientos
    categorias_sentimientos = ['Amenaza', 'Pesimismo', 'Inestabilidad', 'Esceptisismo', 'Indiferente',
            'Neutral', 'Favorable', 'Optimismo', 'Convicción', 'Consolidado', 'Confianza']

    # 1. Diccionario de mapeo
    categorias_sentimientos = {
      'Amenaza': 1,
      'Pesimismo': 2,
      'Inestabilidad': 3,
      'Escepticismo': 4,
      'Indiferente': 5,
      'Neutral': 6,
      'Favorable': 7,
      'Optimismo': 8,
      'Convicción': 9,
      'Consolidado': 10,
      'Confianza': 11
    }

    for frase in frases:
        scores = sia.polarity_scores(frase)
        polaridad = scores['compound']
        distancias = np.abs(pnst - polaridad)
        categoria = np.where(distancias == np.min(distancias))[0][0]
        nd[categoria] += 1

 # Get the key (sentimiento) based on index
        sentimiento = list(categorias_sentimientos.keys())[categoria]

 # 2. Usar el diccionario para obtener el número de categoría
        categoria_num = categorias_sentimientos[sentimiento]

        resultados.append([frase, polaridad, year, categoria_num, sentimiento])

    return resultados, nd  # Mover la declaración return fuera del bucle


# Archivos a analizar (reemplazar con las rutas correctas en Drive)
archivos = {
    "2021": "/content/InformeJuntaDirectiva_2021.txt",
    "2022": "/content/InformeJuntaDirectiva_2022.txt",
    "2023": "/content/InformeJuntaDirectiva_2023.txt",
    "2024": "/content/InformeJuntaDirectiva_2024.txt"
}

# Análisis de todos los años
resultados_totales = []
nd_totales = np.zeros((11, len(archivos)))

for i, (year, archivo) in enumerate(archivos.items()):
    resultados, nd = analizar_sentimientos(archivo, year)
    resultados_totales.extend(resultados)
    nd_totales[:, i] = nd[:, 0]

4. Se procede con la construcción del informe en Excel

In [5]:
# Construcción del informe en Excel
columnas = ['Amenaza', 'Pesimismo', 'Inestabilidad', 'Esceptisismo', 'Indiferente',
            'Neutral', 'Favorable', 'Optimismo', 'Convicción', 'Consolidado', 'Confianza']

df_sentimientos = pd.DataFrame(nd_totales.transpose(), columns=columnas)
df_sentimientos.to_excel("Sentimientos Categorias.xlsx", index=False)

# Guardar resultados completos
df = pd.DataFrame(resultados_totales, columns=['Frase', 'Polaridad','Año', 'Categoría', 'Sentimientos'])
df.to_excel("Análisis de Sentimientos.xlsx", index=False)